### Задача K "Волшебная перестановка".

Ученикам магической школы тоже приходится иметь дело с числами и массивами чисел. Одна из тем занятий – массивы- перестановки. Перестановка – это массив, составленный из $n$ различных целых чисел от $1$ до $n$ в произвольном порядке. Например, $[2,3,1,5,4]$ – перестановка, а $[1,2,2]$ **не** перестановка (2 встречается 2 раза), $[1,3,4]$ тоже не перестановка ($n = 3$, но в массиве является число $4$). Последовательность $а$ является подотрезком $b$, если $а$ может быть получена из $b$ удалением нескольких (возможно ни одного, или всех) элементов с начала и нескольких (возможно ни одного, или всех) с конца. Обозначим подотрезок как $[q, r]$, где $q, r$ – целые числа и $q ≤ r ≤ n$. Это будет подотрезок, из которого изъяли $q - 1$ элементов слева и $n - r$ справа. Для перестановки $p_1, p_2, ..., p_n$ рамочным подотрезком будет такой подотрезок из индексов $[q, r]$, в котором   
$max {pq, pq + 1, ..., pr} – min {pq, pq + 1, ..., pr} = r – q$.   
Например, в перестановке $(6,7,1,8,5,3,2,4)$ некоторые из рамочных подотрезков $[1,2]$, $[5,8]$, $[6,7]$, $[3,3]$, $[8 , 8]$. Так, подотрезок $[i, i]$ всегда является рамочным подотрезком для любого $i$ от $1$ до $n$ включительно. Для перестановки $p$ определим магический потенциал перестановки как количество таких пар $(q, r)$, что $1 ≤ q ≤ r ≤ n$ и $[q, r]$ является рамочным подотрезком. Например, перестановка $[3,1,2]$ имеет магический потенциал 5: все подотрезки индексов, кроме $[1,2]$ являются рамочными. Есть два целых числа $n$ и $m$. Гермиона хочет найти суммарный магический потенциал всех перестановок длины $n$ по модулю простого числа $m$.
Обратите внимание, что всего существует $n!$ различных перестановок длины $n$.

#### 1. Решение полным перебором.

1. Переберем все перестановки длины $n$ с помощью функции permutations из библиотеки itertools&  
2. Для каждой такой перестановки рассмотрим все срезы $[i: j + 1]$.  
3. Для каждого среза $[i:j + 1]$ проверим, является ли он рамочным, если "да", увеличим на единицу магический потенциал текущей перестановки и суммарный магический потенциал всех перестановок длины $n$.  


In [1]:
from itertools import permutations


n, m = map(int, input().split())

ans = 0
for p in permutations([i for i in range(1, n + 1)]):
    for i in range(n):
        for j in range(i, n):
            if max(p[i:j + 1]) - min(p[i:j + 1]) == j - i:
                ans += 1
                ans %= m

print(ans % m)

3 993244853
32


#### 2. Построение комбинаторного решения.

#### 2.1 Избавимся от цикла по p:

In [3]:
ans = 0
for i in range(n):
        for j in range(i, n):
            pass # ans += количество перестановок, в которых good(i, j)

| 1 | 2 | 3 | 4 |  5 | 6 | 7 | 8 | 9 | 10 | 11 | 12 | 13 | 14 | 15 |
|:-:|:-:|:-:|:-:|:--:|:-:|:-:|:-:|:-:|:--:|:--:|:--:|:--:|:--:|:--:|
| 5 | 3 | 2 | 4 | 15 | 6 | 7 | 8 | 9 | 10 | 11 | 13 | 14 |  1 | 12 |

$i = 6$  - левая граница среза  
$j = 11$ - правая граница среза  
$len = j - i + 1 = 6$ - длина среза

Нужно найти ответы на следующие вопросы:  
1. Сколько существует различных наборов из $len$ элементов для заполнения среза $[i:j + 1]$, таких, что $good(i, j) = True$?
2. Сколько существует расстановок такого набора элементов внутри среза $[i:j + 1]$?  
3. Солько существует расстановок оставшихся элементов перестановки для заполнения срезов $[1:i]$ и $[j + 1: n + 1]$?

#### Ответы:

1. $(n - len + 1)$
2. $len!$
3. $(n - len)!$

По правилу умножения найдем количество всех перестановок, у которых $good(i, j)$:  
  
  $\left (n - len + 1  \right ) \cdot len! \cdot \left (n - len  \right )!$

In [7]:
from math import factorial as f

ans = 0
for i in range(n):
        for j in range(i, n):
            len_ = j - i + 1
            ans += (f(len_) * f(n - len_) * (n - len_ + 1))
            ans %= m
            
print(ans % m)

32


Избавились от третьего цикла, но решение всё равно остается медленным.

#### 2.2 Избавимся от двойного цикла по i  и j, заменив их циклом по len:

In [9]:
ans = 0
for len_ in range(1, n + 1):
    ans += (f(len_) * f(n - len_) * (n - len_ + 1) * (n - len_ + 1))
    ans %= m

print(ans % m)

32


Нехорошо каждый раз вычислять факториал с помощью библиотечной функции. Нужно за линию сделать предподсчет:

In [8]:
f = [0] * (n + 2)
f[0] = 1

for i in range(1, n + 1):
    f[i] = f[i - 1] * i

In [9]:
ans = 0
n, m = map(int, input().split())
for len_ in range(1, n + 1):
    ans += (f[len_] * f[n - len_] * (n - len_ + 1) * (n - len_ + 1))
    ans %= m

print(ans % m)

2019 993244853
923958830


In [ ]:
b = set()
for a1 in range(n + 1):
    for a5 in range(n + 1 - a1):
        for a10 in range(n + 1 - a1 - a5):
            a50 = n - a1 - a5 - a10
            if a50 >= 0:
                x = a1 + 5 * a5 + 10 * a10 + 50 * a50
                b.add(x)
                    
print(len(b))
    

In [ ]:
for x in range(n, 50 * n + 1):
    if good(x):
        ans += 1